Для выполнения заданий возьмите данные из папки [data](https://drive.google.com/drive/folders/17E4vB5kWr5FNjLKRU4XGoJSVTxn1jkg4?usp=sharing) о продажах. Данные распределены по 4 файлам:
* sales_train.csv - данные о продажах с января 2013 по октябрь 2015.
** date - дата продажы
** date_block_num - номер месяца по порядку следования
** shop_id - идентификатор магазина
** item_id - идентификатор товара
** item_price - цена товара
** item_cnt_day - количество проданного товара одного вида в день

* items.csv - подробная информация о товарах
** item_id - идентификатор товара
** item_name - название товара
** category_id - идентификатор категории товара

* item_categories.csv - подробная информация о категориях товаров
** item_category_id - идентификатор категории товара
** category_name - название категории товара

* shops.csv - подробная информация о магазинах
** shop_id - идентификатор магазина
** shop_name - название магазина


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
item_cat = pd.read_csv('/content/item_categories.csv')
items = pd.read_csv('/content/items.csv')
sales_train = pd.read_csv('/content/sales_train.csv', parse_dates = ['date'])
shops = pd.read_csv('/content/shops.csv')

<ipython-input-91-393e04e12236>:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  sales_train = pd.read_csv('/content/sales_train.csv', parse_dates = ['date'])


In [ ]:
item_cat.rename(columns={'item_category_id':'category_id'}, inplace=True)

# Задание 1 (3 балла)

1. Возьмите 10 000 случайных строк из таблицы sales_train и найдите в них долю товаров из категории 37. 
2. Найдите самый удачный по общему количеству продаж день для магазина St. Petersburg Nevsky Center shopping center за 2014 год.
3. Есть ли товар, который продается успешнее всех остальных (по ежемесячному количеству) во всех магазинах?

**1.1**

In [ ]:
df_rand = sales_train.sample(10000)
df_m = df_rand.merge(items, how='left', on='item_id')
df_m = df_m['category_id'].value_counts(normalize = True).reset_index()
df_m.columns = ['category_id', 'fraction']
df_m[df_m.category_id == 37]

,category_id,fraction
4,37,0.0684


**1.2**

In [ ]:
df1_m =sales_train.merge(shops, how='left', on='shop_id')
df1_m['year'] = df1_m['date'].dt.year
df1_m[(df1_m['year'] == 2014) &(df1_m['shop_name'] == ' St. Petersburg Nevsky Center shopping center')]\
.groupby('date', as_index = False)\
.agg({'item_cnt_day':'sum'})\
.nlargest(1,'item_cnt_day')

,date,item_cnt_day
360,2014-12-27,588.0


**1.3**

In [ ]:
df1_m['ym'] = df1_m['date'].dt.to_period('M')
df1_m.groupby(['ym','shop_name','item_id'], as_index=False)\
.agg({'item_cnt_day':'sum'}).sort_values(['ym','shop_name','item_cnt_day'],ascending=[True,True,False]).groupby(['ym','shop_name']).head(1)

,ym,shop_name,item_id,item_cnt_day
134,2013-01,"Adygea TC ""Mega""",4164,9.0
610,2013-01,"Balashikha TC ""Oktyabr-Kinomir""",2308,7.0
2502,2013-01,"Chekhov TC"" Karnaval """,19811,24.0
2637,2013-01,Digital warehouse 1C-Online,492,11.0
3110,2013-01,Internet-shop of emergency situations,11370,73.0
...,...,...,...,...
1798546,2015-12,"Yakutsk Ordzhonikidze, 56",20949,18.0
1798988,2015-12,"Yakutsk TC"" Central """,20949,13.0
1799245,2015-12,"Yaroslavl TC"" Altair """,20949,11.0
1799270,2015-12,Zhukovsky st. Chkalov 39m?,3007,8.0


# Задание 2 (2 балла)

1. Найдите топ-10 самых часто покупаемых товаров (не забудьте учесть столбец item_cnt_day).
2. Оставьте только те из них, которые покупали все три года (2013, 2014, 2015).

**2.1**

In [ ]:
df_si = sales_train.merge(items, how = 'left', on = 'item_id')
df_si['year'] = df_si['date'].dt.year


In [ ]:
df_10_rank = df_si.groupby('item_name', as_index = False ).agg({'item_cnt_day':'sum','year':'nunique'})

In [ ]:
df_10_rank['rank'] = df_10_rank['item_cnt_day'].rank(method='min',ascending = False)

In [ ]:
df_10_rank[(df_10_rank['rank']<=10)].sort_values('item_cnt_day', ascending = False)

,item_name,item_cnt_day,year,rank
5473,Corporate package T-shirt 1C Interest white (3...,187642.0,3,1.0
6278,"Diablo III [PC, Jewel, Russian version]",17245.0,3,2.0
8796,"Grand Theft Auto V [PS3, Russian subtitles]",16642.0,3,3.0
1906,Acceptance of funds for 1C-Online,15830.0,3,4.0
14818,Playstation Store replenishment of wallet: Pay...,14515.0,3,5.0
8798,"Grand Theft Auto V [Xbox 360, Russian subtitles]",11688.0,3,6.0
17943,Sony PlayStation 4 (500 Gb) Black (CUH-1008A/1...,10289.0,3,7.0
8795,"Grand Theft Auto V [PC, Russian subtitles]",10099.0,1,8.0
3466,"Battlefield 4 [PC, Russian version]",10032.0,3,9.0
17900,"Some of us [PS3, Russian version]",9227.0,3,10.0


# **2.2**

In [ ]:
df_10_rank[(df_10_rank['rank']<=10)&(df_10_rank['year']==3)].sort_values('item_cnt_day', ascending = False)

,item_name,item_cnt_day,year,rank
5473,Corporate package T-shirt 1C Interest white (3...,187642.0,3,1.0
6278,"Diablo III [PC, Jewel, Russian version]",17245.0,3,2.0
8796,"Grand Theft Auto V [PS3, Russian subtitles]",16642.0,3,3.0
1906,Acceptance of funds for 1C-Online,15830.0,3,4.0
14818,Playstation Store replenishment of wallet: Pay...,14515.0,3,5.0
8798,"Grand Theft Auto V [Xbox 360, Russian subtitles]",11688.0,3,6.0
17943,Sony PlayStation 4 (500 Gb) Black (CUH-1008A/1...,10289.0,3,7.0
3466,"Battlefield 4 [PC, Russian version]",10032.0,3,9.0
17900,"Some of us [PS3, Russian version]",9227.0,3,10.0


# Задание 3 (3 балла)

> При выполнении на pandas не используйте merge().

1. Определите категорию товаров, в которой совершается минимальное число покупок. Выведите айди и название категории.
2. Определите категорию товаров, в которой совершается максимальное число покупок. Выведите айди и название категории.

*Если таких категорий несколько, выведите все.*

In [ ]:
items_copy = items.copy()
items_copy = items_copy.set_index('item_id')['category_id']
sales_train_copy = sales_train.copy()
sales_train_copy = sales_train_copy.set_index('item_id')
df_m = sales_train_copy.assign(category_id = items_copy)

In [ ]:
dd_cat = df_m.groupby('category_id').agg({'item_cnt_day':'sum'})


In [ ]:
dd_cat['rnk_l'] = dd_cat['item_cnt_day'].rank(method='dense')

In [ ]:
dd_cat['rnk_h'] = dd_cat['item_cnt_day'].rank(method='dense',ascending=False)

In [ ]:
dff_min = dd_cat[dd_cat['rnk_l'] == 1]

In [ ]:
dff_max = dd_cat[dd_cat['rnk_h'] == 1]

In [ ]:
dff_max

,item_cnt_day,rnk_l,rnk_h
category_id,,,
40,634171.0,80.0,1.0


In [ ]:
item_cat = item_cat.set_index('category_id')

In [ ]:
dff_max = dff_max.assign(name_cat = item_cat)

In [ ]:
dff_min = dff_min.assign(name_cat = item_cat)

In [152]:
dff_min = dff_min.reset_index()[['category_id','name_cat']]

In [153]:
dff_min

,category_id,name_cat
0,10,Game consoles - PS2
1,51,Books - Cognitive literature


In [ ]:
dff_max =dff_max.reset_index()[['category_id','name_cat']]

In [154]:
dff_max

,category_id,name_cat
0,40,Cinema - DVD


In [ ]:
cat_max = (dff_max.iloc[:,0].to_list())
print(cat_max)

[40]


In [ ]:
cat_min = dff_min.iloc[:,0].to_list()
print(cat_min)

[10, 51]


In [ ]:
cat = cat_max + cat_min

In [ ]:
cat

[40, 10, 51]

# Задание 4 (3 балла)

1. Для категорий из задания 3 найдите среднее количество продаж каждого товара категории в 5 магазинах с наибольшими продажами. Оставьте в таблице только те товары, среднее количество продаж которых строго больше 100.
2. Найдите месяц с максимальными продажами для каждого товара из пункта 1.


# **4.1**

In [ ]:
sim = sales_train.merge(items)

In [ ]:
si = sim[sim['category_id'] == 40].groupby(['item_id','shop_id']).agg({'item_cnt_day':'sum'}).reset_index()

In [ ]:
si = si.groupby('item_id')['item_cnt_day'].nlargest(5).groupby('item_id').mean().reset_index()

In [ ]:
items_from_5avg = si[si['item_cnt_day']>100]['item_id'].to_list()

In [ ]:
items_from_5avg

[30,
 32,
 8415,
 8778,
 9781,
 9862,
 9983,
 10292,
 10298,
 10376,
 10487,
 10497,
 10956,
 10991,
 11457,
 11496,
 11921,
 11922,
 11966,
 12361,
 12362,
 12734,
 13811,
 14019,
 14305,
 14447,
 14574,
 14730,
 16450,
 16454,
 16501,
 16581,
 16676,
 17782,
 18622,
 19065,
 19116,
 19196,
 19602,
 19771,
 19811,
 19900,
 20088,
 20231,
 21376,
 21377,
 21384,
 21404,
 21427,
 21440,
 21876,
 21880,
 22020,
 22058]

In [ ]:
sim['ym'] = sim['date'].dt.to_period('M')

In [ ]:
dsc = sim[sim['item_id'].isin(items_from_5avg)]

In [ ]:
dsc

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,category_id,ym
427712,2013-01-13,0,23,21876,299.0,1.0,MONSTER HIGH. CLASSIC GIRLS,40,2013-01
427713,2013-01-30,0,21,21876,299.0,1.0,MONSTER HIGH. CLASSIC GIRLS,40,2013-01
427714,2013-01-19,0,27,21876,299.0,1.0,MONSTER HIGH. CLASSIC GIRLS,40,2013-01
427715,2013-10-01,0,28,21876,299.0,1.0,MONSTER HIGH. CLASSIC GIRLS,40,2013-10
427716,2013-01-27,0,28,21876,299.0,1.0,MONSTER HIGH. CLASSIC GIRLS,40,2013-01
...,...,...,...,...,...,...,...,...,...
2690219,2015-10-17,33,25,19116,449.0,1.0,GUARANTEES OF GALAXY,40,2015-10
2690220,2015-10-28,33,25,19116,449.0,1.0,GUARANTEES OF GALAXY,40,2015-10
2690221,2015-10-10,33,28,19116,399.0,1.0,GUARANTEES OF GALAXY,40,2015-10
2690222,2015-10-21,33,28,19116,449.0,1.0,GUARANTEES OF GALAXY,40,2015-10


# **4.2**

In [ ]:
dsc.groupby(['item_id','ym'], as_index=False).agg({'item_cnt_day':'sum'}).sort_values(['item_id','item_cnt_day'], ascending=[True,False]).groupby('item_id').head(1)

,item_id,ym,item_cnt_day
1,30,2013-02,872.0
35,32,2013-01,190.0
73,8415,2014-03,484.0
89,8778,2013-05,122.0
129,9781,2013-09,609.0
150,9862,2014-03,386.0
180,9983,2013-10,402.0
204,10292,2013-09,300.0
242,10298,2013-11,467.0
268,10376,2013-12,368.0


# Задание 5 (3 балла)

1. Для категорий из задания 3 найдите среднюю и медианную цену по категории в пределах каждого месяца. Выведите топ-10 товаров по медиане из каждой категории.
2. В каждом году найдите месяцы, в которых средняя цена меньше, чем в предыдущем и следующем месяце.

# **5.1**

In [ ]:
d3 = sales_train.merge(items, on='item_id', how='left')

In [ ]:
d3['ym'] = d3['date'].dt.to_period('M')

In [ ]:
d3[d3['category_id'].isin(cat)].groupby(['ym','category_id']).agg({'item_price':['mean','median']})

item_price        
                            mean  median
ym      category_id                     
2013-01 40            246.840006   199.0
2013-02 40            251.290121   199.0
2013-03 40            256.452211   199.0
2013-04 10           1998.000000  1998.0
        40            251.836481   199.0
2013-05 40            244.512712   199.0
2013-06 40            247.805791   199.0
2013-07 40            241.600264   198.0
2013-08 40            241.169040   198.9
2013-09 40            246.931731   199.0
2013-10 40            247.326370   199.0
2013-11 40            250.554545   199.0
2013-12 40            254.302370   199.0
2014-01 40            258.085791   205.0
2014-02 40            260.827884   239.0
2014-03 40            259.905347   237.0
2014-04 40            265.300205   235.0
2014-05 40            257.785476   199.0
2014-06 40            259.013657   199.0
2014-07 40            256.064176   199.0
        51            129.000000   129.0
2014-08 40            264.093259   229.0
2014-09 40            261.882816   221.0
2014-10 40            259.143043   199.0
2014-11 40            267.371134   230.0
2014-12 40            259.515229   199.0
2015-01 40            269.665403   249.0
2015-02 40            262.484604   199.0
2015-03 40            264.725962   199.0
2015-04 40            278.017070   249.0
2015-05 40            279.956755   249.0
2015-06 40            272.406096   199.0
2015-07 40            267.406448   199.0
2015-08 40            275.428186   249.0
2015-09 40            273.062226   249.0
2015-10 40            275.000777   249.0
2015-11 40            271.959038   249.0
2015-12 40            275.298937   249.0

In [ ]:
d3[d3['category_id'].isin(cat)].groupby(['category_id','item_name']).agg({'item_price':'median'}).reset_index().groupby('category_id')['item_price'].nlargest(10).to_frame()

item_price
category_id                 
10          0        1998.00
40          1541     1399.00
            4109     1299.00
            2558      999.00
            3135      899.00
            4783      899.00
            2935      898.85
            495       849.00
            499       849.00
            513       849.00
            533       849.00
51          4955      129.00

# **5.2**

In [ ]:
d3['m'] = d3['date'].dt.month
d3['y'] = d3['date'].dt.year

In [ ]:
df2013 = d3[d3['y'] == 2013]
df2013 = df2013.groupby('m').agg({'item_price':'mean'})
df2013['next'] = df2013['item_price'].shift(-1)
df2013['prev'] = df2013['item_price'].shift(1)
df2013[(df2013['item_price'] < df2013['next']) & (df2013['item_price'] < df2013['prev'])]

,item_price,next,prev
m,,,
4,671.825293,678.349465,702.466212
7,646.497804,661.772593,680.952738
10,746.258608,786.016631,755.543642


In [ ]:
df2014 = d3[d3['y'] == 2014]
df2014 = df2014.groupby('m').agg({'item_price':'mean'})
df2014['next'] = df2014['item_price'].shift(-1)
df2014['prev'] = df2014['item_price'].shift(1)
df2014[(df2014['item_price'] < df2014['next']) & (df2014['item_price'] < df2014['prev'])]

,item_price,next,prev
m,,,
4,892.942940,915.651651,948.009894
7,886.182415,904.793464,907.177535
10,1005.476975,1170.851105,1052.227452


In [ ]:
df2015 = d3[d3['y'] == 2015]
df2015 = df2015.groupby('m').agg({'item_price':'mean'})
df2015['next'] = df2015['item_price'].shift(-1)
df2015['prev'] = df2015['item_price'].shift(1)
df2015[(df2015['item_price'] < df2015['next']) & (df2015['item_price'] < df2015['prev'])]

,item_price,next,prev
m,,,
2,1043.389266,1077.597099,1096.251765
4,1074.749596,1096.765782,1077.597099
7,963.194169,979.920239,1018.833670


# **ЗАЩИТА ЛАБЫ ВАРИАНТ 19**

**Найдите самый дорогой товар в категории 3 в 2013 и 2015 году.**

In [ ]:
df = sales_train.merge(items)

In [ ]:
df['y'] = df['date'].dt.year

In [ ]:
df_2013 = df[(df['category_id'] == 3) & (df['y']==2013) ]

In [ ]:
df_2013['rnk_h'] = df_2013['item_price'].rank(method='dense',ascending=False)

<ipython-input-138-af0ad3866ef3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2013['rnk_h'] = df_2013['item_price'].rank(method='dense',ascending=False)


In [ ]:
df_2013 =df_2013[df_2013['rnk_h'] == 1]

In [ ]:
df_2013.drop_duplicates(subset=['item_id'],keep='first')

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,category_id,y,rnk_h
2018253,2013-12-14,11,26,5671,2690.0,1.0,PS4: Controller game wireless blue (Dualshock ...,3,2013,1.0
2020152,2013-09-12,11,25,5672,2690.0,1.0,PS4: Game controller wireless black (Dualshock...,3,2013,1.0
2139256,2013-09-12,11,26,5669,2690.0,1.0,PS4: Game controller wireless red (Dualshock 4...,3,2013,1.0


In [ ]:
df2015 = df[(df['category_id'] == 3) & (df['y']==2015)]

In [ ]:
df2015['rnk_h'] = df2015['item_price'].rank(method='dense',ascending=False)

<ipython-input-142-c3bc016d3508>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2015['rnk_h'] = df2015['item_price'].rank(method='dense',ascending=False)


In [ ]:
df2015 = df2015[df2015['rnk_h'] == 1]

In [ ]:
df2015.drop_duplicates(subset=['item_id'],keep='first')

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,category_id,y,rnk_h
2391781,2015-04-23,27,28,5657,7490.0,1.0,PS4: Wireless Black Headset for PS4 (with PS3 ...,3,2015,1.0


**Найдите топ-5 товаров по количеству продаж в каждом декабре датасета.**


In [ ]:
df['ym'] = df['date'].dt.to_period('M')

In [ ]:
df_2013_12 = df[df['ym'] == '2013-12']

In [ ]:
df_2013_12.groupby('item_name').agg({'item_cnt_day':'count'}).nlargest(5,'item_cnt_day')

,item_cnt_day
item_name,
Corporate package T-shirt 1C Interest white (34 * 42) 45 microns,1047
"Battlefield 4 [PC, Russian version]",562
"Grand Theft Auto V [PS3, Russian subtitles]",550
"Grand Theft Auto V [Xbox 360, Russian subtitles]",508
"Assassin's Creed IV. Black flag [PC, Jewel, Russian version]",486


In [ ]:
df_2014_12 = df[df['ym'] == '2014-12']

In [ ]:
df_2014_12.groupby('item_name').agg({'item_cnt_day':'count'}).nlargest(5,'item_cnt_day')

,item_cnt_day
item_name,
Corporate package T-shirt 1C Interest white (34 * 42) 45 microns,1260
Acceptance of funds for 1C-Online,707
"Sims 4 [PC, Russian version]",555
"Grand Theft Auto V [PS4, Russian subtitles]",474
"FIFA 15 [PS3, Russian version]",473


In [ ]:
df_2015_12 = df[df['ym'] == '2015-12']

In [ ]:
df_2015_12.groupby('item_name').agg({'item_cnt_day':'count'}).nlargest(5,'item_cnt_day')

,item_cnt_day
item_name,
Corporate package T-shirt 1C Interest white (34 * 42) 45 microns,270
Acceptance of funds for 1C-Online,131
Playstation Store replenishment of wallet: Payment card 1000 rubles.,75
"Grand Theft Auto V [PC, Russian subtitles]",70
"Sims 4 [PC, Russian version]",67
